In [ ]:
import sys
import numpy as np
import matplotlib.pyplot as plt
import cebra
from PIL import Image
import cv2
import torch
import pickle
import os

In [ ]:
brain_path = '2020_11_9_MV1_run_brain'
behavior_path = '2020_11_9_MV1_run_behavior'
feature_label_path = 'feature_labels'
cebra_model_path = 'cebra_multi_model2.pt'
output_folder_path = 'output'

In [ ]:
def process_brain(brain_seq):
  try:
    brain_seq = np.array(brain_seq)
    brain_mask = (np.sum(brain_seq, axis=0) > 0)
    flat_seq = (brain_seq[:, brain_mask])
    return flat_seq.astype(float)
  except:
    print(np.shape(brain_seq))

def import_data(filepath, processor, max = -1):
    output_data = []
    output_name = []
    for iter, file in enumerate(os.listdir(filepath)):
     filename = os.fsdecode(file)
     if filename.endswith(".tif"):
         out = cv2.imreadmulti(filepath + '/' + filename)[1]
         output_data.append(processor(out))
         output_name.append(filename)
         if iter > max and max > 0: break
         continue
     else:
         continue
    return output_data, output_name

def flatten_data(data):
    data_flat = np.squeeze(data[0])
    for x in data[1::]:
        data_flat = np.concatenate((data_flat, np.squeeze(x)))
    return data_flat

def generate_CEBRA_embeddings(neural, name, model, session = 'run'):
    embedding = []
    failed = []
    for run, data in enumerate(neural):
        try:
            if session == 'run':
                embedding.append(model.transform(data, session_id=run))
            else:
                embedding.append(model.transform(data, session_id=1))
        except:
            failed.append(run)
            print(run)
    failed.reverse()
    for index in failed:
        del name[index]
        del neural[index]
    return embedding

In [ ]:
# Load brain and behavior data
brain_data, name_data = import_data(brain_path, process_brain)
behavior_data, _ = import_data(behavior_path, lambda x: x)

In [ ]:
#load feature labels
with open(feature_label_path, 'rb') as f:
    feature_labels = pickle.load(f)

In [ ]:
#get neural embeddings using cebra model
model = cebra.CEBRA.load(cebra_model_path)
neural_embeddings = generate_CEBRA_embeddings(brain_data, name_data, model, session = 'run')

In [ ]:
#sepaarate into train and predict groups, and flatten data
train_cutoff = int(len(neural_embeddings) * 0.8)
train_neural_embeddings = neural_embeddings[:train_cutoff]
train_names = name_data[:train_cutoff]
train_behavior = behavior_data[:train_cutoff]
train_feature_labels = feature_labels[:train_cutoff]
train_neural_embeddings_flat = flatten_data(train_neural_embeddings)
train_behavior_flat = flatten_data(train_behavior)
train_feature_labels_flat = flatten_data(train_feature_labels)

predict_neural_embeddings = neural_embeddings[train_cutoff:]
predict_names = name_data[train_cutoff:]
predict_behavior = behavior_data[train_cutoff:]
predict_neural_embeddings_flat = flatten_data(predict_neural_embeddings)
predict_behavior_flat = flatten_data(predict_behavior)

In [ ]:
#fit kNN regressor to flattened training data
image_decoder = cebra.KNNDecoder(n_neighbors=20, metric="cosine")
image_decoder.fit(train_neural_embeddings_flat, train_feature_labels_flat)


In [ ]:
#predict feature label vector from neural embedding
predict_feature_labels_flat = image_decoder.predict(predict_neural_embeddings_flat)

In [ ]:
def normalize_array(in_array):
    return np.array([x / np.linalg.norm(x) for x in in_array])

def match_frame_to_embeddings(predicted_embedding, embedding_train, image_train):
  cos_dist = np.matmul(embedding_train, predicted_embedding.T)
  index_list = np.argmax(cos_dist, axis=0)
  return image_train[index_list]

In [ ]:
#normalize predicted feature label vector and train feature label vector\
predict_feature_labels_flat = normalize_array(predict_feature_labels_flat)
train_feature_labels_flat = normalize_array(train_feature_labels_flat)

In [ ]:
# generate series of predicted images
predicted_frames = match_frame_to_embeddings(predict_feature_labels_flat, train_feature_labels_flat, train_behavior_flat)

In [ ]:
#run Agglomerative clustering to predict class labels
from sklearn.cluster import AgglomerativeClustering
clustering = AgglomerativeClustering(n_clusters=3).fit(predict_neural_embeddings_flat)
predicted_labels_flat = clustering.labels_

In [ ]:
def reshape_frames(frames, shape_ref):
    shape_list = [np.shape(x)[0] for x in shape_ref]
    gen_video_list = []
    index = 0
    for shape in shape_list:
        gen_video_list.append((frames[index : index + shape]))
        index += shape
    return gen_video_list

def display_frames_as_video(frames, ground_truth, frame_rate, name, labels):
    fontScale = 1
    org = (50, 50)
    font = cv2.FONT_HERSHEY_SIMPLEX
    thickness = 2
    # Get the dimensions of the frames
    frame_height, frame_width = frames[0].shape

    # Create a VideoWriter object to write the frames into a video file
    video_writer = cv2.VideoWriter(output_folder_path + name +'.mp4',
                                   cv2.VideoWriter_fourcc(*'mp4v'),
                                   frame_rate,
                                   (frame_width, 2 * frame_height))

    # Display frames
    for iter, frame in enumerate(zip(frames, ground_truth, labels)):
        # Write the current frame to the video file
        color_frame = cv2.cvtColor(((frame[0]/2 + frames[iter - 1]/2)).astype(np.uint8), cv2.COLOR_GRAY2RGB)
        color_truth = cv2.cvtColor(frame[1].astype(np.uint8), cv2.COLOR_GRAY2RGB)
        combined = np.concatenate((color_frame, color_truth))

        # write corresponding label to video corner
        combined  = cv2.putText(combined, str(frame[2]), org, font, 
                   fontScale, (0,0,255), thickness, cv2.LINE_AA)
        video_writer.write(combined)

    # Release the VideoWriter and close the window
    video_writer.release()
    cv2.destroyAllWindows()